# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [89]:
import pandas as pd
import numpy as np
from scipy import stats

In [90]:
# Read data
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

# Store white, black, and combined interview calls
white = data[data.race=='w']
black = data[data.race=='b']
calls = data.call.values

## Q1: Appropriate Test and Central Limit Theorem

We want to test if there is a significant relationship between `applicant race` and `interview offer`. Namely, we want to see if there is a difference between the proportion of interview offers given to `white` and `black` candidates.

We will perform this test in two ways:
1. `Bootstrap approach - Permutation`
2. `Frequentist approach - Two Sample Z Test`

Do we meet CLT requirements?
1. `Randomness`: will be assumed by the data.
2. `Normal`: since we are looking at proportions, we need to verify we have at least `10` successes and `10` failures

In [91]:
print('Black: Successes =', sum(black.call), ', Failures =', len(black)-sum(black.call))
print('White: Successes =', sum(white.call), ', Failures =', len(white)-sum(white.call))

Black: Successes = 157.0 , Failures = 2278.0
White: Successes = 235.0 , Failures = 2200.0


3. `Independence`: we have `2435` names from each race, which is much smaller than the entire population. Using the `10% rule`, we can assume independence.

## Q2: Null and Alternate Hypotheses

`H0: P1 - P2 = 0`

`Ha: P1 - P2 != 0`

`α = 0.05`

Here `P1` and `P2` represent the proportion of offers given to white and black candidates respectively.

## Q3: Conduct Statistical Tests

### 1. Bootstrap Approach

In [92]:
# Compute size of samples so we don't have to recompute them
sample_size = len(calls)
half_sample_size = int(sample_size/2)

# Compute difference between white and black call proportions
proportion_diff = np.abs(sum(white.call) - sum(black.call)) / half_sample_size

# Generate 10000 permutations of all calls. For each permutation, assign first half
# as white, second half as black and, and store difference of interview proportions.
proportion_diffs = np.empty(10000)
for i in range(10000) :
    rand_perm = np.random.permutation(calls)
    white_perm = rand_perm[:half_sample_size]
    black_perm = rand_perm[half_sample_size:]
    proportion_diffs[i] = ( sum(white_perm) - sum(black_perm) ) / half_sample_size

# Count num differences as extreme as observed proportion difference.
# Multiply by 2 for two tailed test.
print('p-value:', 2 * np.sum(proportion_diffs >= proportion_diff) / 10000)

p-value: 0.0


Repeating the experiment above multiple times, `I continuously got p-values that were zero or way below our alpha value of 0.05`. So based on this test, we would `reject the Null Hypothesis` and conclude that there is a relationship between `applicant race` and `interview offer`.

### 2. Frequentist Approach

In [93]:
# Compute proportions: white, black, both
white_prop = sum(white.call) / half_sample_size
black_prop = sum(black.call) / half_sample_size
prop = (sum(white.call) + sum(black.call)) / sample_size

# Compute standard error, margin of error, and z stat: 
standard_error = np.sqrt( (2*prop*(1-prop))/half_sample_size )
z_critical = 1.96
margin_of_error = z_critical * standard_error
z_stat = (white_prop - black_prop) / standard_error

# Calculate p-value. Two tailed test
# Use negative z_stat for cdf.
p_value = stats.norm.cdf(-z_stat) * 2 

# Print Z stat and p-value
print('z-stat: ', z_stat)
print('p_value: ', p_value)
print('margin of error:', margin_of_error)
print('95% confidence interval: (', proportion_diff - margin_of_error, ',', proportion_diff + margin_of_error, ')')

z-stat:  4.108412152434346
p_value:  3.983886837585077e-05
margin of error: 0.015281912310894095
95% confidence interval: ( 0.01675094189855149 , 0.04731476652033968 )


Again, our `p-value is almost zero`, so just like with the bootstrap test, `we reject the Null Hypothesis` and conclude that there is a significant relationship between `applicant race` and `interview offer`.

Another way to look at it is to say that our confidence interval for the difference in proportions is entirely above `0`, which yeilds the same conclusion as above.

## Q4: Statistical Summary

Confining our analysis to the data provided, we conclude that there is a statistical difference between `white` and `black` names and whether or not they receive interview offers. White applicants are `50%` more likley to land an interview than black applicants, which is a significant difference.

Racial discrimination has been prevalent for a long period of time, and not only in the employment sector. One idea would be to repeat this experiment accross multiple years and see how the difference in proportions fluctuates.

## Q5: Further Analysis 

Our data indicates that `race` is an important factor when it comes to recieving an `interview offer`. Nethertheless, there are other factors which need to be considered such as `education`, `years of experience`, `address`, etc, which all could play a significant role.